## Getting testing data coordinates from S3

In this notebook we will download a dataframe populated with testing data.

However, all values will be blank (NaN).

We expect competitors to run their predictive models and fill in the blank locations
using their predictions at the IL/XL/TWT locations.

There are two blind wells for evaluation.

Once results are submitted, we will be calculating R^2 scores with ground truth data
and results will be put on the leaderboard.

Doing imports.

In [2]:
from pandas import read_json

## Loading AWS Credentials

See Tutorials #1 for setting up credentials on local machines.

## Getting the Blank DataFrame

We have the blind wells in the same format as training wells.

They have inline, crossline, and two-way time values provided (coordinates).

We expect you to run your feature extraction, feature engineering, and predictions around or at
these locations and populate this DataFrame without adding/removing or changing the shape.

We will then evaluate results comparing these to ground truth data.

See the output of this cell for what it looks like.

**Any result that is not the same shape as this DataFrame will not be considered.**

**Any result that has `NaN` values in the results DataFrame will not be considered.**

In [3]:
well_bucket = 's3://sagemaker-gitc2021/poseidon/wells/'
well_file = 'poseidon_geoml_testing_wells_blank.json.gz'

well_df = read_json(
    path_or_buf=well_bucket + well_file,
    compression='gzip',
)

well_df.set_index(['well_id', 'twt'], inplace=True)

well_df

index     inline      xline  rhob  p_impedance  s_impedance
well_id twt                                                                
well_07 2708.0      0  3256.5233  1389.3570   NaN          NaN          NaN
        2708.5      1  3256.5234  1389.3558   NaN          NaN          NaN
        2709.0      2  3256.5236  1389.3546   NaN          NaN          NaN
        2709.5      3  3256.5237  1389.3534   NaN          NaN          NaN
        2710.0      4  3256.5238  1389.3522   NaN          NaN          NaN
...               ...        ...        ...   ...          ...          ...
well_13 3245.0   1783  2407.1965  2675.8730   NaN          NaN          NaN
        3245.5   1784  2407.1917  2675.8885   NaN          NaN          NaN
        3246.0   1785  2407.1866  2675.9051   NaN          NaN          NaN
        3246.5   1786  2407.1811  2675.9232   NaN          NaN          NaN
        3247.0   1787  2407.1751  2675.9426   NaN          NaN          NaN

[1788 rows x 6 columns]

Below are the statistics. As you can see, the rhob, p_impedance, and s_impedance are blank.

In [4]:
well_df.describe()

,index,inline,xline,rhob,p_impedance,s_impedance
count,1788.000000,1788.000000,1788.000000,0.0,0.0,0.0
mean,893.500000,2997.641068,1781.104954,NaN,NaN,NaN
std,516.295458,391.072115,591.295179,NaN,NaN,NaN
min,0.000000,2407.175100,1389.007600,NaN,NaN,NaN
25%,446.750000,2408.253850,1389.064000,NaN,NaN,NaN
50%,893.500000,3256.583700,1389.222750,NaN,NaN,NaN
75%,1340.250000,3257.007125,2671.146450,NaN,NaN,NaN
max,1787.000000,3257.313500,2675.942600,NaN,NaN,NaN


We will accept results uploaded into these S3 buckets.

Please use following paths and file names as a template.

**Intermediate:**

`bucket =` *`s3://sagemaker-gitc2021/poseidon/wells/intermediate/`*

`file_name =` *`TeamName_Intermediate_Results_YYYYMMDD.json.gz`*

**Final:**

`bucket =` *`s3://sagemaker-gitc2021/poseidon/wells/final/`*

`file_name = `*`TeamName_Final_Results_YYYYMMDD.json.gz`*

Final submissions must be in the same `.json.gz` format. This can be achieved by using the
following code snippet. This assumes your populated DataFrame variable is named `result`.

`bucket =` *`see_links_above`*

`file_name =` *`see_format_above`*
```
result.to_json(
    path_or_buf=bucket + file_name,
    double_precision=4,
    compression='gzip'
)
```